# Custom scorer

In [1]:
import pandas as pd

Let's create a dummy "wine" dataset of 1000 wines, 10 features, and 3 classes (0=bad, 1=medium, 2=good wine)

In [2]:
from sklearn.datasets import make_classification

X, y = make_classification(
    n_samples=1000, n_features=10, n_classes=3, n_clusters_per_class=1, weights=[0.6, 0.3, 0.1], random_state=0
)

In [3]:
pd.DataFrame(X)

,0,1,2,3,4,5,6,7,8,9
0,-0.149077,0.853148,-0.111843,-0.101805,0.698096,0.135308,-0.210313,1.068731,-0.233866,-0.061851
1,-0.032801,0.563769,-0.362429,-0.724720,-0.471613,-0.754443,0.302274,0.395243,-0.936288,-0.868138
2,0.668339,-0.479297,0.263382,0.108647,-0.953179,0.299120,-0.014197,-0.759934,-0.472323,0.491435
3,-2.405761,0.773981,0.741387,0.742029,0.742540,0.528902,0.218237,0.398010,-0.222363,1.208082
4,1.103842,-0.690178,-1.200861,-0.711177,0.749876,-0.774795,-0.421027,0.018713,0.255672,-1.910952
...,...,...,...,...,...,...,...,...,...,...
995,0.238571,-1.145754,0.768922,0.847658,-0.463588,-0.462465,1.060124,0.834569,-1.115651,0.687056
996,0.788629,-0.517723,0.006231,0.149472,1.193810,-0.069265,0.062623,-0.788068,-1.440632,-0.031104
997,-1.369391,1.281511,-0.556269,0.323357,-0.766328,1.886736,-2.047014,-1.012673,-1.027297,0.371755
998,-1.178617,0.552980,0.752483,0.378860,0.758996,2.120077,-1.084214,-1.312394,1.066272,2.112363


In [4]:
pd.Series(y).value_counts()

0    599
1    299
2    102
dtype: int64

-----
❓ Our objective is to train a model which **maximizes prediction precision for the good wines (y=2) only**.  

We don't want any customers to be dissatisfied!

----

## Optimizing for accuracy 

In [5]:
# Split train/test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 0)

❓ Random Search CV the best SVC model measured on the `accuracy`
(you can limit yourself to `rbf` kernels to start with)

In [18]:
%%time
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats

# Instanciate model
model = SVC()

# Hyperparameter search space
search_space = {
    'kernel': ['poly'],
    'C': stats.uniform(0.01, 1000),
    'degree': stats.randint(1,10),
}

# Instanciate Random Search
search = RandomizedSearchCV(
    model, search_space,
    n_jobs=-1, scoring='accuracy', cv=5, n_iter=100, verbose=1,random_state = 0)

# fit it
search.fit(X_train, y_train)

# show best model
search.best_estimator_

Fitting 5 folds for each of 100 candidates, totalling 500 fits
CPU times: user 466 ms, sys: 27.8 ms, total: 494 ms
Wall time: 2.95 s


SVC(C=778.1667509498504, degree=1, kernel='poly')

In [19]:
search.best_params_

{'C': 778.1667509498504, 'degree': 1, 'kernel': 'poly'}

❓ Print classification report on the test set

In [8]:
from sklearn.metrics import classification_report
print(classification_report(y_test, search.best_estimator_.predict(X_test)))

              precision    recall  f1-score   support

           0       0.91      0.97      0.94       183
           1       0.91      0.78      0.84        88
           2       0.69      0.69      0.69        29

    accuracy                           0.89       300
   macro avg       0.84      0.81      0.82       300
weighted avg       0.89      0.89      0.88       300



☝️ Not good enough, we want to focus **only** on class 2

## Custom scoring function

Let's make our own custom metric which returns the precision of class "2" 

reminder: ```precision2 = TP2/(TP2+FP2)``` 

In [41]:
def my_custom_metric(y_true, y_pred):
    
     # Initiate count of predicted 2s and actual 2s
    predicted_2 = 0
    actually_2 = 0
    
    # Go through predictions and true values
    for (idx, y_p) in enumerate(y_pred):
        if y_p ==2:
            predicted_2 +=1 #If prediction = 2, count +1
            if y_true[idx] == 2:
                actually_2 +=1 #If true value is also 2, count +1
                
    if predicted_2 == 0: # Incase there are no 2s predicted, return zero to avoid division / zero
        return 0 
    else:
        return actually_2/predicted_2 # Otherwise, return precision

_true = [0,0,1,2,2,1]
_pred = [0,1,1,2,1,0]

my_custom_metric(_true, _pred) # should return 1

1.0

In [10]:
def my_custom_metric_2(y_true, y_pred):
    
    # Initiate count of predicted 2s and actual 2s
    predicted_2 = 0
    actually_2 = 0
    
    # Go through predictions and true values
    for (idx, y_p) in enumerate(y_pred):
        if y_p ==2:
            predicted_2 +=1 #If prediction = 2, count +1
            if y_true[idx] == 2:
                actually_2 +=1 #If true value is also 2, count +1
                
    if predicted_2 == 0: # Incase there are no 2s predicted, return zero to avoid division / zero
        return 0 
    else:
        return actually_2/predicted_2 # Otherwise, return precision


_true = [0,0,1,2,2,1]
_pred = [0,1,1,2,1,0]

my_custom_metric_2(_true, _pred)  # should return 1

1.0

## Optimizing hyper-params for our custom scoring function

In [20]:
# Can you use this function in  sklearn (will crash)
RandomizedSearchCV(SVC(), search_space, 
                   n_iter = 100,
                   cv=5,
                   scoring=my_custom_metric,
                   random_state = 0)\
.fit(X_train, y_train)

/Users/bingobango/.pyenv/versions/3.10.6/envs/tom/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/bingobango/.pyenv/versions/3.10.6/envs/tom/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
TypeError: my_custom_metric() takes 2 positional arguments but 3 were given

  warnings.warn(
/Users/bingobango/.pyenv/versions/3.10.6/envs/tom/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/bingobango/.pyenv/versions/3.10.6/envs/tom/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    score

/Users/bingobango/.pyenv/versions/3.10.6/envs/tom/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/bingobango/.pyenv/versions/3.10.6/envs/tom/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
TypeError: my_custom_metric() takes 2 positional arguments but 3 were given

  warnings.warn(
/Users/bingobango/.pyenv/versions/3.10.6/envs/tom/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/bingobango/.pyenv/versions/3.10.6/envs/tom/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    score

/Users/bingobango/.pyenv/versions/3.10.6/envs/tom/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/bingobango/.pyenv/versions/3.10.6/envs/tom/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
TypeError: my_custom_metric() takes 2 positional arguments but 3 were given

  warnings.warn(
/Users/bingobango/.pyenv/versions/3.10.6/envs/tom/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/bingobango/.pyenv/versions/3.10.6/envs/tom/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    score

/Users/bingobango/.pyenv/versions/3.10.6/envs/tom/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/bingobango/.pyenv/versions/3.10.6/envs/tom/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
TypeError: my_custom_metric() takes 2 positional arguments but 3 were given

  warnings.warn(
/Users/bingobango/.pyenv/versions/3.10.6/envs/tom/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/bingobango/.pyenv/versions/3.10.6/envs/tom/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    score

/Users/bingobango/.pyenv/versions/3.10.6/envs/tom/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/bingobango/.pyenv/versions/3.10.6/envs/tom/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
TypeError: my_custom_metric() takes 2 positional arguments but 3 were given

  warnings.warn(
/Users/bingobango/.pyenv/versions/3.10.6/envs/tom/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/bingobango/.pyenv/versions/3.10.6/envs/tom/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    score

...

/Users/bingobango/.pyenv/versions/3.10.6/envs/tom/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/bingobango/.pyenv/versions/3.10.6/envs/tom/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
TypeError: my_custom_metric() takes 2 positional arguments but 3 were given

  warnings.warn(
/Users/bingobango/.pyenv/versions/3.10.6/envs/tom/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/bingobango/.pyenv/versions/3.10.6/envs/tom/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    score

/Users/bingobango/.pyenv/versions/3.10.6/envs/tom/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/bingobango/.pyenv/versions/3.10.6/envs/tom/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
TypeError: my_custom_metric() takes 2 positional arguments but 3 were given

  warnings.warn(
/Users/bingobango/.pyenv/versions/3.10.6/envs/tom/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/bingobango/.pyenv/versions/3.10.6/envs/tom/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    score

/Users/bingobango/.pyenv/versions/3.10.6/envs/tom/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/bingobango/.pyenv/versions/3.10.6/envs/tom/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
TypeError: my_custom_metric() takes 2 positional arguments but 3 were given

  warnings.warn(
/Users/bingobango/.pyenv/versions/3.10.6/envs/tom/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/bingobango/.pyenv/versions/3.10.6/envs/tom/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    score

/Users/bingobango/.pyenv/versions/3.10.6/envs/tom/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/bingobango/.pyenv/versions/3.10.6/envs/tom/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
TypeError: my_custom_metric() takes 2 positional arguments but 3 were given

  warnings.warn(
/Users/bingobango/.pyenv/versions/3.10.6/envs/tom/lib/python3.10/site-packages/sklearn/model_selection/_search.py:953: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan

RandomizedSearchCV(cv=5, estimator=SVC(), n_iter=100,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x13e26e110>,
                                        'degree': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x13e0dfaf0>,
                                        'kernel': ['poly', 'rbf']},
                   random_state=0,
                   scoring=<function my_custom_metric at 0x13e28da20>)

### `Make_scorer`

We need to transform our "metric" into a "sklearn scorer method"

In [14]:
from sklearn.metrics import make_scorer

my_custom_scorer = make_scorer(my_custom_metric_2)

### RandomizedSearchCV with custom scorer

In [20]:
search2 = RandomizedSearchCV(SVC(), search_space, 
                   n_iter = 100,
                   cv=5,
                   scoring=my_custom_scorer,
                   random_state = 0)

In [21]:
search2.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=SVC(), n_iter=100,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x134235360>,
                                        'degree': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x134234bb0>,
                                        'kernel': ['poly']},
                   random_state=0, scoring=make_scorer(my_custom_metric_2))

❓ print classification report on the test set and compare with previous one

In [22]:
print(classification_report(y_test, search2.best_estimator_.predict(X_test)))

              precision    recall  f1-score   support

           0       0.69      0.97      0.81       183
           1       0.94      0.34      0.50        88
           2       0.73      0.28      0.40        29

    accuracy                           0.72       300
   macro avg       0.79      0.53      0.57       300
weighted avg       0.77      0.72      0.68       300



✅  We improved our precision for class 2 from 0.69 up to 0.73, but at the detriment of overall accuracy!